### Download images from dataset

In [ ]:
import requests
import os

def download_artwork(artwork_name, artist_name):
    save_dir = '../data/artworks/'
    os.makedirs(save_dir, exist_ok=True)
    base_url = "https://www.wikiart.org/en/api/2/PaintingSearch"

    params = {
        'term': f"{artwork_name} {artist_name}",
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if 'data' in data and data['data']:
        # assume the first result is the desired artwork
        artwork = data['data'][0]
        image_url = artwork['image']
        title = artwork['title']
        artist = artwork['artistName']
        file_name = f"{artist} - {title}.jpg"
        file_path = os.path.join(save_dir, file_name)

        img_response = requests.get(image_url)
        if img_response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(img_response.content)
            print(f"Artwork downloaded successfully: {file_path}")
        else:
            print(f"Failed to download image. Status code: {img_response.status_code}")
    else:
        print("No artwork found with the given name and artist.")

### Using Google Lens

In [ ]:
import io
import os
from google.cloud import vision

def detect_labels(image_path):
    client = vision.ImageAnnotatorClient()

    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations

    print('Labels:')
    for label in labels:
        print(label.description)

    if response.error.message:
        raise Exception(f'{response.error.message}')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/your/service-account-file.json"

In [ ]:
image_path = "path/to/your/drawing.jpg"
detect_labels(image_path)

In [ ]:
import io
import os
from google.cloud import vision
from google.cloud.vision_v1 import types
from google.api_core.client_options import ClientOptions

def detect_labels(image_path):
    client_options = ClientOptions(api_key=api_key)
    client = vision.ImageAnnotatorClient(client_options=client_options)

    # Load the image from the file
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    # Perform label detection on the image
    response = client.label_detection(image=image)
    labels = response.label_annotations

    print('Labels:')
    for label in labels:
        print(label.description)

    if response.error.message:
        raise Exception(f'{response.error.message}')

In [ ]:
def detect_web_entities(image_path):
    client_options = ClientOptions(api_key=api_key)
    client = vision.ImageAnnotatorClient(client_options=client_options)

    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    response = client.web_detection(image=image)
    web_detection = response.web_detection

    entities = []
    artwork_name = ""

    print('Web Entities:')
    for entity in web_detection.web_entities:
        entities.append(entity.description)
        print(f'{entity.description} (Score: {entity.score})')

    print('Best Guess Labels:')
    for label in web_detection.best_guess_labels:
        entities.append(label.label)
        print(label.label)

    if response.error.message:
        raise Exception(f'{response.error.message}')
    
    artwork_name  = web_detection.web_entities[0].description
    artist_name_score = web_detection.web_entities[0].score
    
    entities_txt = " ".join(entities)
    return entities_txt, artwork_name, artist_name_score


In [ ]:
print(os.listdir("../input/images/images/Edgar_Degas/"))

In [ ]:
import matplotlib.pyplot as plt

image_path = "../input/images/images/Vincent_van_Gogh/Vincent_van_Gogh_5.jpg"
img = plt.imread(image_path)
plt.imshow(img)

image_path = "../input/images/images/Rembrandt/Rembrandt_151.jpg"
img = plt.imread(image_path)
plt.imshow(img)

In [ ]:
print(os.listdir("../data/original_artworks"))

In [ ]:
# should load this to refresh content of the folder
folder_path = os.listdir("../data/original_artworks")
folder_path

In [ ]:
# iterate folder path and calc how many images contain "Vincent_van_Gogh"
arts_per_artists = {"Vincent_van_Gogh": 0, "Edgar_Degas": 0, "Rembrandt": 0, "Pablo_Picasso" : 0 , "Alfred_Sisley": 0, "Titian": 0, "Pierre-Auguste_Renoir": 0, "Paul_Gauguin": 0, "Francisco_Goya": 0}

# iterate all the names of the artworks in the folder and search if name contains the artist name
for img_name in folder_path:
    if "Vincent_van_Gogh" in img_name:
        arts_per_artists["Vincent_van_Gogh"] += 1
    elif "Edgar" in img_name:
        arts_per_artists["Edgar_Degas"] += 1
    elif "Rembrandt" in img_name:
        arts_per_artists["Rembrandt"] += 1
    elif "Pablo_Picasso" in img_name:
        arts_per_artists["Pablo_Picasso"] += 1
    elif "Alfred_Sisley" in img_name:
        arts_per_artists["Alfred_Sisley"] += 1
    elif "Titian" in img_name:
        arts_per_artists["Titian"] += 1
    elif "Pierre-Auguste_Renoir" in img_name:
        arts_per_artists["Pierre-Auguste_Renoir"] += 1
    elif "Paul_Gauguin" in img_name:
        arts_per_artists["Paul_Gauguin"] += 1
    elif "Francisco_Goya" in img_name:
        arts_per_artists["Francisco_Goya"] += 1

In [ ]:
# print the number of images per artist
print(arts_per_artists)
print("total:" , sum(arts_per_artists.values()))

In [ ]:
# create a csv file eith the following columns: "artist_name", "artwork_name", "image_path", "google_entities", "generated_artwork_name"
import pandas as pd

data = []

for img_name in folder_path:
    artist_name = ""
    if "Vincent_van_Gogh" in img_name:
        artist_name = "Vincent_van_Gogh"
    elif "Edgar" in img_name:
        artist_name = "Edgar_Degas"
    elif "Rembrandt" in img_name:
        artist_name = "Rembrandt"
    elif "Pablo_Picasso" in img_name:
        artist_name = "Pablo_Picasso"
    elif "Alfred_Sisley" in img_name:
        artist_name = "Alfred_Sisley"
    elif "Titian" in img_name:
        artist_name = "Titian"
    elif "Pierre-Auguste_Renoir" in img_name:
        artist_name = "Pierre-Auguste_Renoir"
    elif "Paul_Gauguin" in img_name:
        artist_name = "Paul_Gauguin"
    elif "Francisco_Goya" in img_name:
        artist_name = "Francisco_Goya"
    data.append([artist_name, img_name, "", "", "", "", "", ""])
    
# create the dataframe
df = pd.DataFrame(data, columns=["artist_name", "image_path", "image_path_named", "artwork_name", "artwork_score", "google_entities", "prompt", "generated_artwork_name"])

In [ ]:
df.head()
df.shape

In [ ]:
def get_artist_name(img_name):
    artist_name = ""
    if "Vincent_van_Gogh" in img_name:
        artist_name = "Vincent_van_Gogh"
    elif "Edgar" in img_name:
        artist_name = "Edgar_Degas"
    elif "Rembrandt" in img_name:
        artist_name = "Rembrandt"
    elif "Pablo_Picasso" in img_name:
        artist_name = "Pablo_Picasso"
    elif "Alfred_Sisley" in img_name:
        artist_name = "Alfred_Sisley"
    elif "Titian" in img_name:
        artist_name = "Titian"
    elif "Pierre-Auguste_Renoir" in img_name:
        artist_name = "Pierre-Auguste_Renoir"
    elif "Paul_Gauguin" in img_name:
        artist_name = "Paul_Gauguin"
    elif "Francisco_Goya" in img_name:
        artist_name = "Francisco_Goya"
    return artist_name
    

In [ ]:
import shutil

artwork_named_path = "../data/original_artworks_named"
folder_path = os.listdir("../data/original_artworks")

### Attach the artwork name to the image and add to the dataframe

In [ ]:
counter = 0
start_counter = 204
for img_name in folder_path:
    if counter < start_counter:
        counter += 1
        continue
    artist_name = get_artist_name(img_name)
    print("artist: ", artist_name, " img_path: ", f"../data/original_artworks/{img_name}")
    img_path = f"../data/original_artworks/{img_name}"

    img = plt.imread(img_path)
    plt.imshow(img)
    web_ent_txt, art_name, art_name_score = detect_web_entities(img_path)
    print("artist: ", artist_name, " img_path: ", img_path, " web_ent: ", web_ent_txt, " art_name: ", art_name)
    
    # change spaces to _ in the artwork name
    art_name = art_name.replace(" ", "_")

    # if art_name already exists in the folder, add a number to the end of the name
    if os.path.exists(f"{artwork_named_path}/{artist_name}_{art_name}.jpg"):
        art_name = f"{art_name}_{counter}"
        
    # remove from art_name the following characters: /, \, :, *, ?, ", <, >, |, \n
    art_name = art_name.replace("/", "")
    art_name = art_name.replace("\\", "")
    art_name = art_name.replace(":", "")
    art_name = art_name.replace("*", "")
    art_name = art_name.replace("?", "")
    art_name = art_name.replace("\"", "")
    art_name = art_name.replace("<", "")
    art_name = art_name.replace(">", "")
    art_name = art_name.replace("|", "")
    art_name = art_name.replace("\n", "")
    

   # copy the image to the new folder but don't delete the old one
    new_img_path = f"{artwork_named_path}/{artist_name}_{art_name}.jpg"
    shutil.copy(img_path, new_img_path)
    print("---------------------------------------")
    print("img_path: ", image_path, "art name: ", art_name, "art name score: ", art_name_score,
          "img_name: ", img_name, "new_img_path: ", new_img_path)
    
    # save the data to the dataframe - change the image path to the new path
    df.loc[df["image_path"] == img_name, "google_entities"] = web_ent_txt
    df.loc[df["image_path"] == img_name, "artwork_name"] = art_name
    df.loc[df["image_path"] == img_name, "artwork_score"] = art_name_score
    df.loc[df["image_path"] == img_name, "image_path_named"] = f"{artwork_named_path}/{artist_name}_{art_name}.jpg"

    print("===================" , counter ,"===================")
    counter += 1

In [ ]:
df.head(10)

In [ ]:
# save the dataframe to a csv file
# df.to_csv("../data/artworks_data.csv", index=False)